In [1]:
!pip install -r requirements.txt

In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(
    file_path="jobs.csv"
)
data = loader.load()
data

[Document(metadata={'source': 'jobs.csv', 'row': 0}, page_content='type: info\ncompany: name\nlocation: \nrole: \nstart_date: \nend_date: \ndescription: Ian Dalton'),
 Document(metadata={'source': 'jobs.csv', 'row': 1}, page_content='type: info\ncompany: email\nlocation: \nrole: \nstart_date: \nend_date: \ndescription: daltioan@gmail.com'),
 Document(metadata={'source': 'jobs.csv', 'row': 2}, page_content='type: info\ncompany: phone\nlocation: \nrole: \nstart_date: \nend_date: \ndescription: +54 9 11 3465-2325'),
 Document(metadata={'source': 'jobs.csv', 'row': 3}, page_content='type: info\ncompany: website\nlocation: \nrole: \nstart_date: \nend_date: \ndescription: https://www.linkedin.com/in/ian-dalton-id/'),
 Document(metadata={'source': 'jobs.csv', 'row': 4}, page_content='type: info\ncompany: address\nlocation: \nrole: \nstart_date: \nend_date: \ndescription: Buenos Aires City, Argentina'),
 Document(metadata={'source': 'jobs.csv', 'row': 5}, page_content='type: education\ncompany

In [3]:
from langchain_openai.chat_models.base import ChatOpenAI

from langchain_core.messages import HumanMessage,AIMessage,SystemMessage,BaseMessage
llm = ChatOpenAI(
            base_url="http://localhost:1234/v1",
            api_key="not_needed",
            model="llama-3.2-1b-instruct"
        )

In [5]:
from openai import OpenAI
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
embeddings = client.embeddings.create(input = [line.page_content for line in data], model = "text-embedding-nomic-embed-text-v1.5")

In [6]:
embeddings.data

[Embedding(embedding=[-0.011237996630370617, -0.01146799884736538, -0.18305650353431702, -0.023126041516661644, 0.022347528487443924, -0.005451188888400793, 0.04241540655493736, -0.030370622873306274, 0.013951527886092663, 0.028412887826561928, -0.005837630946189165, 0.00581752136349678, 0.05595126003026962, -0.024702614173293114, 0.056693751364946365, -0.03779580071568489, -0.004820405039936304, -0.07202494889497757, -0.010125094093382359, 0.02243678644299507, 0.017653079703450203, -0.016896089538931847, -0.034783270210027695, 0.020322363823652267, 0.03372322767972946, 0.007576096802949905, 0.013085138984024525, -0.028502892702817917, 0.0013695743400603533, -0.018934018909931183, 0.05086894705891609, 0.04911411553621292, 0.02015017718076706, -0.015549630858004093, -0.01751035638153553, 0.005674791522324085, 0.019942866638302803, 0.08067119866609573, -0.03578994795680046, -0.02640184946358204, 0.03062654286623001, 0.004072181880474091, -0.002189215272665024, -0.01133888028562069, 0.061

In [7]:
import chromadb
from langchain_core.tools import BaseTool
from pydantic import Field, BaseModel


class ChromaTool(BaseTool):
    name: str = "chroma_rag"
    description: str = "Look up the user information in the chroma database"
    response_format: str = "json"

    collection_name: str = Field(..., description="The name of the Chroma collection",default_factory="jobs")
    chroma_client: chromadb.Client = Field(
        default_factory=chromadb.Client, description="The Chroma database client"
    )

    

    def __init__(self,collection_name: str):
        # Initialize attributes after Pydantic's initialization
        super().__init__()
        self.chroma_client = chromadb.Client()
        try:
            # Attempt to create a new collection
            self.collection = self.chroma_client.create_collection(collection_name)
            # Assuming `data` is defined elsewhere, ensure it is passed or available
            self.collection.add(
                documents=[line.page_content for line in data],
                ids=[str(i) for i in range(len(data))]
            )
        except Exception:
            # Fallback to getting an existing collection
            self.collection = self.chroma_client.get_collection(self.collection_name)

    def _run(self, query_text):
        # Query the collection and return results
        return self.collection.query(query_texts=[query_text], n_results=2)


# Instantiate the tool
#tool = ChromaTool(collection_name="jobs")

s:\Github\BetterResume\.conda\Lib\site-packages\pydantic\_internal\_generate_schema.py:775: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `Settings` to V2.
  warn(


SchemaError: Invalid Schema:
model.schema.function-before.schema.model-fields.fields.collection_name.schema.default.default_factory.callable
  Input should be callable [type=callable_type, input_value='jobs', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/callable_type
model.schema.function-before.schema.model-fields.fields.collection_name.schema.default.default_factory.callable
  Input should be callable [type=callable_type, input_value='jobs', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/callable_type

In [15]:
from typing import Any
from langchain.tools import BaseTool
import chromadb
from chromadb.config import Settings
from pydantic import Field, PrivateAttr

class ChromaDBTool(BaseTool):
    name: str = "ChromaDBTool"
    description: str = (
        "A tool that interfaces with a ChromaDB database to retrieve information. "
        "It initializes a connection to a specified collection and runs queries against it."
    )
    collection_name: str = Field(default="default_collection")
    persist_directory: str = Field(default="./chroma_db")

    # Declare private attributes for runtime-only properties.
    _client: Any = PrivateAttr()
    _collection: Any = PrivateAttr()

    def __init__(self, **data):
        super().__init__(**data)
        self._client = chromadb.Client(Settings(persist_directory=self.persist_directory))
        try:
            self._collection = self._client.get_collection(name=self.collection_name)
        except Exception as e:
            self._collection = self._client.create_collection(name=self.collection_name)
            self._collection.add(
                documents=[line.page_content for line in data],
                ids=[str(i) for i in range(len(data))]
            )

    def _run(self, query: str, **kwargs):
        """
        Synchronous method to query the ChromaDB collection.
        Expects a text query and returns the query results.
        """
        try:
            results = self._collection.query(query_texts=[query],n_results=2)
            return results
        except Exception as e:
            return f"Error querying the database: {e}"

    async def _arun(self, query: str, **kwargs):
        """
        Asynchronous version of the _run method.
        Currently, this calls the synchronous version for simplicity.
        """
        return self._run(query, **kwargs)
tool = ChromaDBTool()

In [ ]:

llm_with_tools = llm.bind_tools([tool])

In [ ]:
tool.invoke("software engineer")

In [ ]:

import pandas as pd
data = pd.read_csv("jobs.csv")
data["start_date"] = pd.to_datetime(data["start_date"], format="%d/%m/%Y")
data["end_date"] = pd.to_datetime(data["end_date"], format="%d/%m/%Y")


In [ ]:
data[data["company"] == "address"]["description"].to_list()[0]

In [ ]:
# Iterate over all type == job

for job in data[data["type"] == "job"].iterrows():
    print(job[1]["company"])
    print(job[1]["description"])
    print("----")

In [ ]:
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

import os

# Create the Word document
document = Document()

document.add_heading(data[data["company"] == "name"]["description"].to_list()[0], 0)

document.add_paragraph(f"{data[data["company"] == "address"]["description"].to_list()[0]} • {data[data["company"] == "phone"]["description"].to_list()[0]}")
document.add_paragraph(f"{data[data["company"] == "email"]["description"].to_list()[0]} • {' • '.join(data[data["company"] == "website"]["description"].to_list())}")

document.add_heading('SKILLS', level=1)
document.add_paragraph("""Data Architecture & Engineering -Proficient in designing and implementing scalable data pipelines, databases, and dashboards. Experienced with Python, SQL, R, and cloud-based solutions.. 
Project & Team Management -Led a team of five in developing a Flutter application for CEITBA, ensuring efficiency and scalability with Firebase and Google Cloud. Experienced in Agile methodologies, including Scrum and Kanban. 
Web Scraping & Automation - Built automated data collection bots for business intelligence at Turbodato, using Azure Functions and Azure SQL to streamline data processing.
Cloud & DevOps - Skilled in cloud services such as Azure, Google Cloud, AWS, and Firebase. Experience in bare metal server administration and deploying scalable solutions..
Visualization & Business Intelligence - Implemented interactive dashboards using Power BI, Looker Studio, and Tableau to facilitate data-driven decision-making for our clients.
Software Development - Expertise in multiple programming languages including Python, SQL, R, Dart, Java, and JavaScript. Developed and integrated new technologies to enhance AI-driven systems.
""")
document.add_heading('EXPERIENCE', level=1)

document.add_heading('EXPERIENCE', level=1)


for _, job in data[data["type"] == "job"].iterrows():
    # Create a paragraph for the job entry
    p = document.add_paragraph()
    
    # Add the company, location, and role
    run = p.add_run(f"{job['company']}, {job['location']} • {job['role']}")
    run.bold = True  # Make it bold if needed

    # Add a tab stop for right alignment
    tab_stop = p.paragraph_format.tab_stops.add_tab_stop(Inches(6.5))  # Adjust width as needed
    tab_stop.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT

    # Add the start-end date aligned to the right
    p.add_run("\t")  # Add a tab character
    p.add_run(f"({job['start_date'].strftime('%m/%Y')}) - {job['end_date'].strftime('%m/%Y') if pd.notnull(job['end_date']) else 'Present'}")

    # Add the job description below
    document.add_paragraph(job["description"])

document.add_heading('EDUCATION AND CERTIFICATIONS', level=1)
for education in data[data["type"] == "education"].iterrows():
    p = document.add_paragraph(f"{education[1]["company"]}, {education[1]["location"]} • {education[1]["description"]}")
    
    document.add_paragraph(f"{education[1]["start_date"]} - {education[1]["end_date"] if education[1]["end_date"] else 'Present'}")


# Save the Word document
docx_path = 'demo.docx'
document.save(docx_path)

# Convert the Word document to PDF
try:
    import comtypes.client

    def convert_to_pdf(docx_path, pdf_path):
        word = comtypes.client.CreateObject('Word.Application')
        doc = word.Documents.Open(os.path.abspath(docx_path))
        doc.SaveAs(os.path.abspath(pdf_path), FileFormat=17)  # 17 is the PDF format
        doc.Close()
        word.Quit()

    pdf_path = 'demo.pdf'
    convert_to_pdf(docx_path, pdf_path)
    print(f"Document successfully saved as {pdf_path}")

except ImportError:
    print("comtypes library is required to convert to PDF. Install it using 'pip install comtypes'")

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

In [ ]:
import json

from langchain_core.messages import ToolMessage




class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}


tool_node = BasicToolNode(tools=[tool])

graph_builder.add_node("tools", tool_node)

In [ ]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break